<a href="https://colab.research.google.com/github/gabrielanatalia/Projects/blob/main/Portfolio_Visualizer_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Portfolio Visualizer**



In [ ]:
# @title Load libraries
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from datetime import date, timedelta
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import fsolve
import itertools
import plotly.graph_objects as go
import yfinance as yf
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
from matplotlib import colors
import statsmodels.formula.api as smf
from dateutil.parser import parse
import pytz
import re
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

!git clone https://github.com/gabrielanatalia/Projects/
%cd /content/Projects
import sys
sys.path.append('/content/Projects')
import port_cons as pc

import warnings
warnings.filterwarnings("ignore")


Cloning into 'Projects'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), 5.09 MiB | 10.06 MiB/s, done.
/content/Projects


In [ ]:
def convert_to_datetime(input_str, parserinfo=None):
    return parse(input_str, parserinfo=parserinfo)

def combine_backtest_data(portfolio_names):
    # combined portfolio returns
    all_port_ret = {}

    for portfolio in portfolio_names:
        perf_df = globals()[f"{portfolio}_perf"]
        ret_col = [col for col in perf_df.columns if col.endswith('_port_ret')]
        all_port_ret[portfolio] = perf_df[ret_col]

    df_all_port_ret = pd.concat(all_port_ret.values(), keys=all_port_ret.keys(), axis=1)
    df_all_port_ret.columns = [col[0] for col in df_all_port_ret.columns]

    # combined portfolio weights
    all_port_weights = {}

    for portfolio in portfolio_names:
        wgt_df = globals()[f"{portfolio}_wgt"]
        # wgt_col = [col for col in wgt_df.columns if col.endswith('_wgt')]
        # all_port_weights[portfolio] = wgt_df[wgt_col]
        all_port_weights[portfolio] = wgt_df

    df_all_port_weights = pd.concat(all_port_weights.values(), keys=all_port_weights.keys(), axis=1)

    # combined portfolio backtest daata
    all_port_bt = {}

    for portfolio in portfolio_names:
        bt_df = globals()[f"{portfolio}_perf"]
        all_port_bt[portfolio] = bt_df

    df_all_port_bt = pd.concat(all_port_bt.values(), keys=all_port_bt.keys(), axis=1)
    return df_all_port_ret, df_all_port_weights, df_all_port_bt

def generate_key_data():
    print('-' * 30, 'KEY SUMMARY', '-' * 30)
    display(perf_summary)

    print('\n')
    print('-' * 30, 'RELATIVE YEARLY RETURNS', '-' * 30)
    display(pc.apply_style_heatmap_ret(relative_perf, subset=relative_perf.columns[1:]))

    print('\n')
    print('-' * 30, 'CUMULATIVE RETURNS', '-' * 30)
    pc.plot_cumulative_returns(all_port_ret, show_data=True)

    print('\n')
    print('-' * 30, 'RETURNS DISTRIBUTION', '-' * 30)
    pc.plot_returns_distribution_boxplot(all_port_ret)

    print('\n')
    print('-' * 30, 'LATEST ALLOCATION', '-' * 30)
    display(df_ports_alloc_latest.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    print('\n')
    print('-' * 30, 'AVERAGE ALLOCATION', '-' * 30)
    display(df_ports_alloc_avg.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    print('\n')
    print('-' * 30, 'PORTFOLIO ALLOCATION', '-' * 30)
    for port in port_names:
        print(port)
        pc.plot_weights_and_turnover(all_port_wgt[port], show_data=True, show_turnover=True, show_rebal=False)

    print('\n')
    print('-' * 30, 'PORTFOLIO DRAWDOWN', '-' * 30)
    pc.plot_drawdowns(all_port_ret, show_data=True)

    print('\n')
    print('-' * 30, 'CORRELATION', '-' * 30)
    pc.plot_correlation_heatmap(all_port_ret)

# Parameters

Input the following parameters:


*   Start date: format - DD/MM/YYYY
*   End date: format - DD/MM/YYYY
*   Underlying tickers, format - ETF1, ETF2, ETF3, ... | e.g. SPY, EFA, QQQ, RSP, IEMG
*   Benchmark ticker format - ETF1, ETF2, ETF3, ... | e.g. ACWI, URTH, SPY
*   Lookback window (days): the number of days after the specified start date used to initialize the optimization process - this ensures enough data is available before the backtest begins (i.e. if lookback window is 365 days and start date is Jan 1 2010, the actual backtest will begin in Jan 1 2011)







In [ ]:
# @title
# Widget for start date
start_date_widget = widgets.DatePicker(
    description='Start Date:',
    value=parse("2010-01-01").date(),  # Default value
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for end date
end_date_widget = widgets.DatePicker(
    description='End Date:',
    value=parse("2024-10-31").date(),  # Default value
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for tickers
tickers_widget = widgets.Text(
    value='SPY, EFA, QQQ, RSP, IEMG',
    description='Tickers:',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for benchmark tickers
bm_tickers_widget = widgets.Text(
    value='ACWI, URTH, SPY',
    description='Benchmark Tickers:',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for backtest offset days
backtest_offset_days_widget = widgets.IntText(
    value=365,
    description='Lookback window (Days):',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Display the widgets
display(start_date_widget)
display(end_date_widget)
display(tickers_widget)
display(bm_tickers_widget)
display(backtest_offset_days_widget)


# Function to update variables based on widget values
def update_variables(change):
    global start_date, end_date, tickers, bm_tickers, all_tickers, backtest_offset_days

    start_date = start_date_widget.value
    end_date = end_date_widget.value

    tickers = re.split(r'\s*,\s*', tickers_widget.value)
    bm_tickers = re.split(r'\s*,\s*', bm_tickers_widget.value)

    all_tickers = list(set(tickers + bm_tickers))

    backtest_offset_days = backtest_offset_days_widget.value


# Observe widget changes and update variables
start_date_widget.observe(update_variables, names='value')
end_date_widget.observe(update_variables, names='value')
tickers_widget.observe(update_variables, names='value')
bm_tickers_widget.observe(update_variables, names='value')
backtest_offset_days_widget.observe(update_variables, names='value')

# Initial variable update
update_variables(None)

DatePicker(value=datetime.date(2010, 1, 1), description='Start Date:', layout=Layout(width='300px'), style=Des…

DatePicker(value=datetime.date(2024, 10, 31), description='End Date:', layout=Layout(width='300px'), style=Des…

Text(value='SPY, EFA, QQQ, RSP, IEMG', description='Tickers:', layout=Layout(width='300px'), style=Description…

Text(value='ACWI, URTH, SPY', description='Benchmark Tickers:', layout=Layout(width='300px'), style=Descriptio…

IntText(value=365, description='Lookback window (Days):', layout=Layout(width='300px'), style=DescriptionStyle…

# Download data

In [ ]:
df_all = yf.download(all_tickers, start=start_date, end=end_date)['Adj Close']
df_all.index = df_all.index.tz_localize(None)
ret_all = df_all.pct_change().dropna()

ret_bm = ret_all[bm_tickers]

earliest_start_date = ret_all.index.min()
start_date_timestamp = pd.Timestamp(start_date)

# Compare earliest_start_date with start_date_timestamp
if earliest_start_date > start_date_timestamp:
    print('\n', 'Earliest start date: ', earliest_start_date)
else:
    pass

[*********************100%***********************]  7 of 7 completed


 Earliest start date:  2012-10-25 00:00:00


# Constituent performance

In [ ]:
# @title Calculate constituent performance
constituent_summary = pc.performance_summary_constituents(ret_all, start_date=start_date, end_date=end_date, frequency='daily')

constituent_performance_button = widgets.Button(description="Generate Constituent Performance", layout=widgets.Layout(width='auto'))

# Use an Output widget to display the results
out = widgets.Output()

def run_constituent_performance(b):
    with out:
        clear_output(wait=True)  # Clear previous output within the Output widget
        constituent_summary = pc.performance_summary_constituents(
            ret_all, start_date=start_date, end_date=end_date, frequency='daily')
        display(constituent_summary)

constituent_performance_button.on_click(run_constituent_performance)

# Display the button and the Output widget
display(constituent_performance_button, out)

Button(description='Generate Constituent Performance', layout=Layout(width='auto'), style=ButtonStyle())

Output()

# Portfolio configuration & parameters

In [ ]:
# @title
TOLERANCE = 1e-10

def _allocation_risk(weights, covariances):

    portfolio_risk = np.sqrt((weights * covariances * weights.T))[0, 0]

    return portfolio_risk

def _assets_risk_contribution_to_allocation_risk(weights, covariances):

    portfolio_risk = _allocation_risk(weights, covariances)

    assets_risk_contribution = np.multiply(weights.T, covariances * weights.T) \
        / portfolio_risk

    return assets_risk_contribution

def _risk_budget_objective_error(weights, args):
    covariances = args[0]
    assets_risk_budget = args[1]
    weights = np.matrix(weights)

    portfolio_risk = _allocation_risk(weights, covariances)

    assets_risk_contribution = \
        _assets_risk_contribution_to_allocation_risk(weights, covariances)

    assets_risk_target = \
        np.asmatrix(np.multiply(portfolio_risk, assets_risk_budget))

    error = sum(np.absolute(assets_risk_contribution - assets_risk_target.T))[0, 0]
    return error

def _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights):

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},{'type': 'ineq', 'fun': lambda x: x})

    optimize_result = minimize(fun=_risk_budget_objective_error,
                               x0=initial_weights,
                               args=[covariances, assets_risk_budget],
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options={'disp': False})

    weights = optimize_result.x
    print(optimize_result.message)
    return weights

def rebal_wgt_riskparity(returns_data, start_date, end_date, rebal_months=[4,10], halflife=3.5, annualized=252, shrink_covar=False):
    num_of_assets = len(returns_data.columns)

    ret_data_filtered = returns_data.loc[start_date:end_date]
    first_date = returns_data.index[0]
    tickers = list(returns_data.columns)

    weights = pd.DataFrame(0, index=ret_data_filtered.index, columns=ret_data_filtered.columns)
    weights.index = pd.to_datetime(weights.index, format='%Y-%m-%d')
    reb_flag = pd.DataFrame(0, index=weights.index, columns=['reb_flag'])

    alpha = 1 - math.exp(math.log(0.5) / (halflife * annualized))
    span = (2 / alpha) - 1

    for i in range(len(ret_data_filtered)):
        curr_date = ret_data_filtered.index[i]
        curr_date_format = curr_date.strftime('%Y-%m-%d')

        if (weights.index[i].month in rebal_months and weights.index[i-1].month != weights.index[i].month) or i==0:

            if shrink_covar:
                shrink_covar_matrix_model = LedoitWolf().fit(returns_data.loc[first_date:curr_date])
                covar_ann = pd.DataFrame(shrink_covar_matrix_model.covariance_ * annualized, index=tickers, columns=tickers)
                covar_ann.columns = tickers
                covar_ann.index = tickers
            else:
                exp_cov_matrix = returns_data.loc[first_date:curr_date].ewm(span=span).cov(pairwise=True).iloc[-num_of_assets:]
                covar_ann = exp_cov_matrix * annualized
                covar_ann.columns = tickers
                covar_ann.index = tickers

            valid_covar_ann = covar_ann.dropna(how='all')
            valid_covar_ann = valid_covar_ann.dropna(axis=1, how='all')
            valid_num_assets = len(valid_covar_ann)
            valid_tickers = valid_covar_ann.columns

            assets_risk_budget = np.ones([valid_num_assets]) / valid_num_assets
            initial_weights = np.ones([valid_num_assets]) / valid_num_assets

            # print(valid_covar_ann)
            new_wgt = _get_risk_parity_weights(valid_covar_ann.values, assets_risk_budget, initial_weights)
            new_wgt = pd.DataFrame(new_wgt).T
            new_wgt.columns = valid_tickers

            for ticker in valid_tickers:
                weights.at[curr_date_format, ticker] = new_wgt[ticker].values

            reb_flag.loc[curr_date_format] = True

        else:
            weights.iloc[i] = weights.iloc[i-1] * (1+ ret_data_filtered.iloc[i].fillna(0))
            weights_sum = weights.iloc[i].sum()
            weights.iloc[i] /= weights_sum
            reb_flag.iloc[i] = False

    weights = weights.rename(columns={c: c + '_wgt' for c in weights.columns})
    weights = pd.concat([reb_flag, weights], axis=1)

    return weights

In [ ]:
# @title

# Portfolio 1 widgets
portfolio_1_tickers = widgets.Text(
    value='SPY, EFA, QQQ, RSP, IEMG',
    description='Tickers:',
    disabled=False
)
portfolio_1_allocation_type = widgets.Dropdown(
    options=['Max sharpe', 'Min vol', 'Risk parity', 'Equal weight'],
    value='Max sharpe',
    description='Allocation:',
    disabled=False,
)
portfolio_1_name = widgets.Text(
    value='P1_Max_sharpe',
    description='Name:',
    disabled=False
)
portfolio_1_min_weight = widgets.FloatText(
    value=0.0,
    description='Min Weight:',
    disabled=False
)
portfolio_1_max_weight = widgets.FloatText(
    value=1.0,
    description='Max Weight:',
    disabled=False
)

# Arrange Portfolio 1 widgets in a layout
display(widgets.HTML("<h3>Portfolio 1</h3>"))
display(portfolio_1_tickers)
display(portfolio_1_allocation_type)
display(portfolio_1_name)
display(portfolio_1_min_weight)
display(portfolio_1_max_weight)

# Portfolio 2 widgets
portfolio_2_tickers = widgets.Text(
    value='SPY, EFA, QQQ, RSP, IEMG',
    description='Tickers:',
    disabled=False
)
portfolio_2_allocation_type = widgets.Dropdown(
    options=['Max sharpe', 'Min vol', 'Risk parity', 'Equal weight'],
    value='Min vol',
    description='Allocation:',
    disabled=False,
)
portfolio_2_name = widgets.Text(
    value='P2_Min_vol',
    description='Name:',
    disabled=False
)
portfolio_2_min_weight = widgets.FloatText(
    value=0.0,
    description='Min Weight:',
    disabled=False
)
portfolio_2_max_weight = widgets.FloatText(
    value=1.0,
    description='Max Weight:',
    disabled=False
)

# Arrange Portfolio 2 widgets in a layout
display(widgets.HTML("<h3>Portfolio 2</h3>"))
display(portfolio_2_tickers)
display(portfolio_2_allocation_type)
display(portfolio_2_name)
display(portfolio_2_min_weight)
display(portfolio_2_max_weight)

# Portfolio 3 widgets
portfolio_3_tickers = widgets.Text(
    value='SPY, EFA, QQQ, RSP, IEMG',
    description='Tickers:',
    disabled=False
)
portfolio_3_allocation_type = widgets.Dropdown(
    options=['Max sharpe', 'Min vol', 'Risk parity', 'Equal weight'],
    value='Risk parity',
    description='Allocation:',
    disabled=False,
)
portfolio_3_name = widgets.Text(
    value='P3_Risk_parity',
    description='Name:',
    disabled=False
)
portfolio_3_min_weight = widgets.FloatText(
    value=0.0,
    description='Min Weight:',
    disabled=False
)
portfolio_3_max_weight = widgets.FloatText(
    value=1.0,
    description='Max Weight:',
    disabled=False
)

# Arrange Portfolio 3 widgets in a layout
display(widgets.HTML("<h3>Portfolio 3</h3>"))
display(portfolio_3_tickers)
display(portfolio_3_allocation_type)
display(portfolio_3_name)
display(portfolio_3_min_weight)
display(portfolio_3_max_weight)

HTML(value='<h3>Portfolio 1</h3>')

Text(value='SPY, EFA, QQQ, RSP, IEMG', description='Tickers:')

Dropdown(description='Allocation:', options=('Max sharpe', 'Min vol', 'Risk parity', 'Equal weight'), value='M…

Text(value='P1_Max_sharpe', description='Name:')

FloatText(value=0.0, description='Min Weight:')

...

Text(value='SPY, EFA, QQQ, RSP, IEMG', description='Tickers:')

Dropdown(description='Allocation:', index=2, options=('Max sharpe', 'Min vol', 'Risk parity', 'Equal weight'),…

Text(value='P3_Risk_parity', description='Name:')

FloatText(value=0.0, description='Min Weight:')

FloatText(value=1.0, description='Max Weight:')

In [ ]:
# @title Confirm portfolio specifications

# Access the values
tickers_1 = re.split(r'\s*,\s*', portfolio_1_tickers.value)
allocation_type_1 = portfolio_1_allocation_type.value
portfolio_name_1 = portfolio_1_name.value
min_weight_1 = portfolio_1_min_weight.value
max_weight_1 = portfolio_1_max_weight.value

tickers_2 = re.split(r'\s*,\s*', portfolio_2_tickers.value)
allocation_type_2 = portfolio_2_allocation_type.value
portfolio_name_2 = portfolio_2_name.value
min_weight_2 = portfolio_2_min_weight.value
max_weight_2 = portfolio_2_max_weight.value

tickers_3 = re.split(r'\s*,\s*', portfolio_3_tickers.value)
allocation_type_3 = portfolio_3_allocation_type.value
portfolio_name_3 = portfolio_3_name.value
min_weight_3 = portfolio_3_min_weight.value
max_weight_3 = portfolio_3_max_weight.value

df_portfolio_specs = pd.DataFrame(portfolio_specs).T

def get_portfolio_specs():
    portfolio_specs = {
        1: {
            'tickers': re.split(r'\s*,\s*', portfolio_1_tickers.value),
            'allocation_type': portfolio_1_allocation_type.value,
            'name': portfolio_1_name.value,
            'min_weight': float(min_weight_1),
            'max_weight': float(max_weight_1)
        },
        2: {
            'tickers': re.split(r'\s*,\s*', portfolio_2_tickers.value),
            'allocation_type': portfolio_2_allocation_type.value,
            'name': portfolio_2_name.value,
            'min_weight': float(min_weight_2),
            'max_weight': float(max_weight_2)
        },
        3: {
            'tickers': re.split(r'\s*,\s*', portfolio_3_tickers.value),
            'allocation_type': portfolio_3_allocation_type.value,
            'name': portfolio_3_name.value,
            'min_weight': float(min_weight_3),
            'max_weight': float(max_weight_3)
        }
    }
    return portfolio_specs

# Define constraints function
def create_constraints(min_weight, max_weight):
    return ({
        'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1
    }, {
        'type': 'ineq', 'fun': lambda weights: weights - min_weight
    }, {
        'type': 'ineq', 'fun': lambda weights: max_weight - weights
    })

# Get portfolio specifications dynamically
portfolio_specs = get_portfolio_specs()

# Create a confirmation button
confirm_button = widgets.Button(description="Confirm Portfolio Specs", layout=widgets.Layout(width='auto'))

# Function to execute on button click
def on_confirm_button_clicked(b):
  display(df_portfolio_specs)
  print("Portfolio specifications confirmed!")
  # You can add any further actions you want to perform here
  # after confirmation, such as starting the optimization.

# Attach the function to the button's on_click event
confirm_button.on_click(on_confirm_button_clicked)

# Display the button
display(confirm_button)

Button(description='Confirm Portfolio Specs', layout=Layout(width='auto'), style=ButtonStyle())

,tickers,allocation_type,name,min_weight,max_weight
1,"[SPY, EFA, QQQ, RSP, IEMG]",Max sharpe,P1_Max_sharpe,0.0,0.3
2,"[SPY, EFA, QQQ, RSP, IEMG]",Min vol,P2_Min_vol,0.0,0.3
3,"[SPY, EFA, QQQ, RSP, IEMG]",Risk parity,P3_Risk_parity,0.0,0.3


Portfolio specifications confirmed!


In [ ]:
# @title Run optimisation

# Create a progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=3, # Total number of portfolios
    description='Optimising:',
    bar_style='',
    style={'bar_color': 'navy'},
    orientation='horizontal'
)

# Display the progress bar
display(progress_bar)

for i in range(1, 4):
    portfolio = portfolio_specs[i]
    portfolio_name = portfolio['name']
    tickers = portfolio['tickers']
    allocation_type = portfolio['allocation_type']
    min_weight = portfolio['min_weight']
    max_weight = portfolio['max_weight']

    # Create constraints for each portfolio based on the min and max weight
    constraints = create_constraints(min_weight, max_weight)

    # Calculate weights based on allocation type:
    if allocation_type == 'Max sharpe':
        globals()[f"{portfolio_name}_wgt"] = pc.max_sharpe_rebal_wgt(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            constraints_dict=constraints,
            shrink_covar=False,
            rebal_months=[1],
            annualized=261,
            halflife=3.5
        )
    elif allocation_type == 'Min vol':
        globals()[f"{portfolio_name}_wgt"] = pc.min_vol_rebal_wgt(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            constraints_dict=constraints,
            shrink_covar=False,
            rebal_months=[1],
            annualized=261,
            halflife=3.5
        )
    elif allocation_type == 'Risk parity':
        globals()[f"{portfolio_name}_wgt"] = rebal_wgt_riskparity(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            rebal_months=[1],
            halflife=3.5,
            annualized=261,
            shrink_covar=False
        )

    # Calculate performance:
    globals()[f"{portfolio_name}_perf"] = pc.calc_port_perf(
        ret_all.loc[earliest_start_date + timedelta(days=backtest_offset_days):end_date][tickers], globals()[f"{portfolio_name}_wgt"], name_prefix=portfolio_name
    )

    # Update the progress bar
    progress_bar.value += 1

clear_output(wait=True) # Clear the output after completion
print("Optimisation complete!")


Optimisation complete!


In [ ]:
# @title Historical weights
def show_historical_weights(b):
       clear_output(wait=True) # Clear previous output
       for i in range(1, 4):
           portfolio = portfolio_specs[i]
           portfolio_name = portfolio['name']
           print(portfolio_name)
           pc.display_rebal_wgt(globals()[f"{portfolio_name}_wgt"])
           print('\n')

hist_weights_button = widgets.Button(description="Show Historical Weights", layout=widgets.Layout(width='auto'))
hist_weights_button.on_click(show_historical_weights)
display(hist_weights_button)


P1_Max_sharpe


,reb_flag,SPY_wgt,EFA_wgt,QQQ_wgt,RSP_wgt,IEMG_wgt
Date,,,,,,
2013-10-25 00:00:00,100.00%,30.00%,10.00%,30.00%,30.00%,0.00%
2014-01-02 00:00:00,100.00%,30.00%,10.00%,30.00%,30.00%,0.00%
2015-01-02 00:00:00,100.00%,30.00%,10.00%,30.00%,30.00%,0.00%
2016-01-04 00:00:00,100.00%,30.00%,10.00%,30.00%,30.00%,0.00%
2017-01-03 00:00:00,100.00%,30.00%,10.00%,30.00%,30.00%,0.00%
2018-01-02 00:00:00,100.00%,30.00%,10.00%,30.00%,30.00%,0.00%
2019-01-02 00:00:00,100.00%,30.00%,10.00%,30.00%,30.00%,0.00%
2020-01-02 00:00:00,100.00%,30.00%,10.00%,30.00%,30.00%,0.00%
2021-01-04 00:00:00,100.00%,30.00%,10.00%,30.00%,30.00%,0.00%




P2_Min_vol


,reb_flag,SPY_wgt,EFA_wgt,QQQ_wgt,RSP_wgt,IEMG_wgt
Date,,,,,,
2013-10-25 00:00:00,100.00%,30.00%,11.00%,29.00%,30.00%,0.00%
2014-01-02 00:00:00,100.00%,30.00%,10.15%,29.85%,30.00%,0.00%
2015-01-02 00:00:00,100.00%,30.00%,29.35%,9.59%,30.00%,1.06%
2016-01-04 00:00:00,100.00%,30.00%,30.00%,10.00%,30.00%,0.00%
2017-01-03 00:00:00,100.00%,30.00%,21.85%,18.15%,30.00%,0.00%
2018-01-02 00:00:00,100.00%,30.00%,26.55%,13.45%,30.00%,0.00%
2019-01-02 00:00:00,100.00%,30.00%,30.00%,5.63%,30.00%,4.37%
2020-01-02 00:00:00,100.00%,30.00%,30.00%,0.88%,30.00%,9.12%
2021-01-04 00:00:00,100.00%,30.00%,30.00%,8.39%,13.28%,18.33%




P3_Risk_parity


,reb_flag,SPY_wgt,EFA_wgt,QQQ_wgt,RSP_wgt,IEMG_wgt
Date,,,,,,
2013-10-25 00:00:00,100.00%,21.94%,19.35%,20.92%,20.64%,17.15%
2014-01-02 00:00:00,100.00%,22.01%,19.38%,21.15%,20.81%,16.65%
2015-01-02 00:00:00,100.00%,21.66%,19.98%,19.70%,20.82%,17.84%
2016-01-04 00:00:00,100.00%,21.47%,20.33%,19.44%,21.16%,17.60%
2017-01-03 00:00:00,100.00%,22.33%,19.39%,19.92%,21.36%,17.00%
2018-01-02 00:00:00,100.00%,22.42%,19.66%,19.55%,21.58%,16.78%
2019-01-02 00:00:00,100.00%,21.51%,21.45%,17.84%,21.85%,17.34%
2020-01-02 00:00:00,100.00%,21.31%,21.87%,17.54%,21.62%,17.67%
2021-01-04 00:00:00,100.00%,20.44%,21.86%,18.93%,19.37%,19.40%


# Key Data

In [ ]:
# @title Generate key data

########################################################################################################################

port_names = []  # Initialize an empty list to store names

# Iterate through the portfolio specifications
for portfolio_number, portfolio_data in portfolio_specs.items():
    # Extract the 'name' attribute and append it to the list
    port_names.append(portfolio_data['name'])

all_port_ret, all_port_wgt, all_port_bt = combine_backtest_data(port_names)
ret_bm_filtered = ret_bm.loc[earliest_start_date + timedelta(days=backtest_offset_days):]
ret_bm_filtered.iloc[0] = 0
all_port_ret = pd.concat([all_port_ret, ret_bm], axis=1).dropna()

########################################################################################################################


perf_summary = pc.performance_summary(all_port_ret[port_names], all_port_wgt, all_port_ret['SPY'], benchmark_returns=ret_bm,
                                    start_date=earliest_start_date + timedelta(days=backtest_offset_days), end_date=end_date, frequency='daily', rebal_per_year=1).T

relative_perf = pc.calculate_relative_per(all_port_ret[port_names], all_port_ret['SPY'])
df_ports_alloc_latest = pc.compare_port_alloc(all_port_wgt, port_names, latest_data=True)
df_ports_alloc_avg = pc.compare_port_alloc(all_port_wgt, port_names, latest_data=False)

port_turnover_df = pd.DataFrame()

for portfolio in port_names:
    turnover = pc.portfolio_turnover(all_port_wgt[portfolio])
    port_turnover_df = pd.concat([port_turnover_df, turnover],axis=1)

port_turnover_df.columns = port_names


def generate_key_data_func(b):
       clear_output(wait=True)  # Clear previous output
       generate_key_data() # Assuming generate_key_data is already defined


key_data_button = widgets.Button(description="Generate Key Data")
key_data_button.on_click(generate_key_data_func)
display(key_data_button)

------------------------------ KEY SUMMARY ------------------------------


,SPY,P1_Max_sharpe,P2_Min_vol,P3_Risk_parity,ACWI,URTH
Cumulative return,300.85%,307.77%,186.71%,197.85%,163.39%,183.64%
Annualized return,13.97%,14.15%,10.43%,10.82%,9.55%,10.31%
1y cumulative return,34.82%,31.10%,25.46%,27.08%,29.17%,30.37%
3y cumulative return,39.65%,33.73%,18.86%,22.71%,24.41%,28.33%
5y cumulative return,120.98%,123.83%,72.24%,90.19%,84.72%,94.23%
8y cumulative return,208.72%,217.92%,129.23%,154.73%,139.61%,153.68%
3y ann. return,11.78%,10.17%,5.93%,7.06%,7.55%,8.67%
5y ann. return,17.18%,17.48%,11.49%,13.72%,13.06%,14.20%
8y ann. return,15.13%,15.56%,10.93%,12.40%,11.54%,12.34%
Annualized volatility,17.37%,17.95%,16.98%,17.36%,16.84%,17.23%




------------------------------ RELATIVE YEARLY RETURNS ------------------------------


,Benchmark_absolute_ret,P1_Max_sharpe,P2_Min_vol,P3_Risk_parity
2013,5.54%,-0.29%,-0.33%,-1.85%
2014,13.46%,-0.09%,-0.13%,-5.34%
2015,1.23%,1.07%,-1.17%,-2.40%
2016,12.00%,-1.81%,-2.95%,-2.88%
2017,21.71%,2.67%,1.74%,4.82%
2018,-4.57%,-0.57%,-2.79%,-3.36%
2019,31.22%,0.71%,-3.56%,-3.41%
2020,18.33%,6.19%,-4.73%,1.57%
2021,28.73%,-1.92%,-10.74%,-9.57%
2022,-18.18%,-1.95%,1.47%,-0.90%




------------------------------ CUMULATIVE RETURNS ------------------------------


...



------------------------------ PORTFOLIO ALLOCATION ------------------------------
P1_Max_sharpe


P2_Min_vol


P3_Risk_parity




------------------------------ PORTFOLIO DRAWDOWN ------------------------------




------------------------------ CORRELATION ------------------------------


# Export data to excel

In [ ]:
# @title Input file name and export key data to excel
# Create a text input widget for the file path
file_path_widget = widgets.Text(
    value='Portfolio_Visualizer_Output.xlsx',
    description='File Name:',
    disabled=False
)

# Display the widget
display(file_path_widget)

# Function to save the data to Excel
def save_to_excel(file_path):
    with pd.ExcelWriter(file_path) as writer:
      constituent_summary.to_excel(writer, sheet_name='Constituents summary', index=True)
      perf_summary.to_excel(writer, sheet_name='Perf summary', index=True)
      # yearly_df.filter(like='_Annual Return').to_excel(writer, sheet_name='Calendar year returns', index=True)
      df_ports_alloc_latest.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Latest alloc', index=True)
      df_ports_alloc_avg.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Latest alloc', index=True)

      # port_alloc_diff_df.to_excel(writer, sheet_name='Relative latest alloc', index=True)
      # country_breakdown_all.pipe(apply_style_heatmap).pipe(codebase.apply_2dp_percentage).to_excel(writer, sheet_name='Country', index=True)
      # sector_breakdown_all.pipe(apply_style_heatmap).pipe(codebase.apply_2dp_percentage).to_excel(writer, sheet_name='Sector', index=True)
      # country_diff_df.to_excel(writer, sheet_name='Country Relative', index=True)
      # sector_diff_df.to_excel(writer, sheet_name='Sector Relative', index=True)
      # port_turnover_df.to_excel(writer, sheet_name='Turnover', index=True)
      # df_factor_exp.to_excel(writer, sheet_name='ETF Factor MSCI', index=True)
      # df_factor_ff_exp.to_excel(writer, sheet_name='ETF Factor FF', index=True)
      # df_factor_port_exp.to_excel(writer, sheet_name='Portfolio Factor MSCI', index=True)
      # df_factor_ff_port_exp.to_excel(writer, sheet_name='Portfolio Factor FF', index=True)
      # exposure_diff.to_excel(writer, sheet_name='Relative Factor MSCI', index=True)
      # exposure_diff_ff.to_excel(writer, sheet_name='Relative Factor FF', index=True)
      for n in port_names:
          all_port_wgt[n].to_excel(writer, sheet_name=n+'_hist_wgt', index=True)

      print(f"Excel file saved to: Projects/Projects/{file_path}")

# Create a button to trigger the save function
save_button = widgets.Button(description="Save as Excel")

# Define the button's on_click behavior
def on_save_button_clicked(b):
    save_to_excel(file_path_widget.value)

save_button.on_click(on_save_button_clicked)

# Display the button
display(save_button)

Text(value='Portfolio_Visualizer_Output.xlsx', description='File Name:')

Button(description='Save as Excel', style=ButtonStyle())